# Plant Disease Detection

In [ ]:
!pip install ultralytics -q
!pip install kagglehub -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.3 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import kagglehub
import os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Preparing the dataset

In [ ]:
!git clone --depth 1 --filter=blob:none --sparse https://github.com/spMohanty/PlantVillage-Dataset.git
%cd PlantVillage-Dataset
!git sparse-checkout set raw/grayscale

Cloning into 'PlantVillage-Dataset'...
remote: Enumerating objects: 204, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 204 (delta 37), reused 202 (delta 37), pack-reused 0 (from 0)
Receiving objects: 100% (204/204), 7.48 MiB | 27.34 MiB/s, done.
Resolving deltas: 100% (37/37), done.
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 40 (delta 28), reused 39 (delta 28), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 276.64 KiB | 4.07 MiB/s, done.
Resolving deltas: 100% (28/28), done.
/content/PlantVillage-Dataset
remote: Enumerating objects: 54284, done.
remote: Counting objects: 100% (54284/54284), done.
remote: Compressing objects: 100% (54284/54284), done.
remote: Total 54284 (delta 0), reused 54284 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (54284/54284), 766.07 MiB | 38.92 MiB/s, done.
Updat

In [ ]:
import os
import shutil
import random

random.seed(50)

# Define paths
dataset_path = "raw/grayscale"  # Original dataset
output_path = "PlantVillage"  # New organized dataset

# Train, Validation, Test split ratios
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Ensure output directories exist
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(output_path, split), exist_ok=True)

# Split dataset
for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    if not os.path.isdir(class_path):
        continue

    images = os.listdir(class_path)
    images = images[:500]
    random.shuffle(images)

    train_split = int(len(images) * train_ratio)
    val_split = int(len(images) * (train_ratio + val_ratio))

    subsets = {
        "train": images[:train_split],
        "val": images[train_split:val_split],
        "test": images[val_split:]
    }

    for split, files in subsets.items():
        split_class_path = os.path.join(output_path, split, class_name)
        os.makedirs(split_class_path, exist_ok=True)

        for file in files:
            shutil.move(os.path.join(class_path, file), os.path.join(split_class_path, file))

# Remove the old dataset folder after splitting
# shutil.rmtree(dataset_path)

print("Dataset successfully split into train, val, and test sets!")


Dataset successfully split into train, val, and test sets!


In [ ]:
print(output_path)

PlantVillage


## Training

In [ ]:
# Load a model
model = YOLO("yolo11n-cls.yaml")  # build a new model from YAML

YOLO11n-cls summary: 86 layers, 1,633,584 parameters, 1,633,584 gradients, 3.3 GFLOPs


In [ ]:
# Train the model
results = model.train(data=output_path, epochs=100, imgsz=256, batch=16, device="cuda")

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.yaml, data=PlantVillage, epochs=100, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

100%|██████████| 5.35M/5.35M [00:00<00:00, 89.5MB/s]


AMP: checks passed ✅


train: Scanning /content/PlantVillage-Dataset/PlantVillage/train... 12534 images, 0 corrupt: 100%|██████████| 12534/12534 [00:02<00:00, 5441.20it/s]


train: New cache created: /content/PlantVillage-Dataset/PlantVillage/train.cache


...


      Epoch    GPU_mem       loss  Instances       Size


    100/100     0.289G     0.1138          6        256: 100%|██████████| 784/784 [01:29<00:00,  8.72it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 111/111 [00:09<00:00, 11.49it/s]


                   all      0.955      0.999

100 epochs completed in 2.852 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 3.3MB
Optimizer stripped from runs/classify/train/weights/best.pt, 3.3MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-cls summary (fused): 47 layers, 1,574,702 parameters, 0 gradients, 3.2 GFLOPs
train: /content/PlantVillage-Dataset/PlantVillage/train... found 12534 images in 38 classes ✅ 
val: /content/PlantVillage-Dataset/PlantVillage/val... found 3549 images in 38 classes ✅ 
test: /content/PlantVillage-Dataset/PlantVillage/test... found 1827 images in 38 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 111/111 [00:09<00:00, 11.50it/s]


                   all      0.955      0.999
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


In [ ]:
# Validate the model
metrics = model.val()
print(metrics)

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-cls summary (fused): 47 layers, 1,574,702 parameters, 0 gradients, 3.2 GFLOPs
train: /content/PlantVillage-Dataset/PlantVillage/train... found 12534 images in 38 classes ✅ 
val: /content/PlantVillage-Dataset/PlantVillage/val... found 3549 images in 38 classes ✅ 
test: /content/PlantVillage-Dataset/PlantVillage/test... found 1827 images in 38 classes ✅ 


val: Scanning /content/PlantVillage-Dataset/PlantVillage/val... 3549 images, 0 corrupt: 100%|██████████| 3549/3549 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 222/222 [00:09<00:00, 24.05it/s]


                   all      0.954      0.999
Speed: 0.1ms preprocess, 1.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2
ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a9ac018af10>
curves: []
curves_results: []
fitness: 0.9766131341457367
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9543533325195312, 'metrics/accuracy_top5': 0.9988729357719421, 'fitness': 0.9766131341457367}
save_dir: PosixPath('runs/classify/train2')
speed: {'preprocess': 0.12149672752896763, 'inference': 1.0468365626896468, 'loss': 0.002608414760265404, 'postprocess': 0.0017294249100552948}
task: 'classify'
top1: 0.9543533325195312
top5: 0.9988729357719421


In [ ]:
# Testing
model.val(split="test")

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
train: /content/PlantVillage-Dataset/PlantVillage/train... found 12534 images in 38 classes ✅ 
val: /content/PlantVillage-Dataset/PlantVillage/val... found 3549 images in 38 classes ✅ 
test: /content/PlantVillage-Dataset/PlantVillage/test... found 1827 images in 38 classes ✅ 


test: Scanning /content/PlantVillage-Dataset/PlantVillage/test... 1827 images, 0 corrupt: 100%|██████████| 1827/1827 [00:00<00:00, 5428.04it/s]

test: New cache created: /content/PlantVillage-Dataset/PlantVillage/test.cache



               classes   top1_acc   top5_acc: 100%|██████████| 115/115 [00:04<00:00, 25.92it/s]


                   all       0.95      0.998
Speed: 0.2ms preprocess, 1.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train3


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a9aac17d390>
curves: []
curves_results: []
fitness: 0.9740010797977448
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9496442079544067, 'metrics/accuracy_top5': 0.9983579516410828, 'fitness': 0.9740010797977448}
save_dir: PosixPath('runs/classify/train3')
speed: {'preprocess': 0.19125130268324161, 'inference': 1.0333095911317942, 'loss': 0.005202834702073431, 'postprocess': 0.004523179531430983}
task: 'classify'
top1: 0.9496442079544067
top5: 0.9983579516410828

## Zip and download the model

In [ ]:
!zip -r /content/model.zip /content/PlantVillage-Dataset/runs

  adding: content/PlantVillage-Dataset/runs/ (stored 0%)
  adding: content/PlantVillage-Dataset/runs/classify/ (stored 0%)
  adding: content/PlantVillage-Dataset/runs/classify/train2/ (stored 0%)
  adding: content/PlantVillage-Dataset/runs/classify/train2/confusion_matrix.png (deflated 15%)
  adding: content/PlantVillage-Dataset/runs/classify/train2/val_batch2_pred.jpg (deflated 1%)
  adding: content/PlantVillage-Dataset/runs/classify/train2/val_batch0_pred.jpg (deflated 1%)
  adding: content/PlantVillage-Dataset/runs/classify/train2/confusion_matrix_normalized.png (deflated 15%)
  adding: content/PlantVillage-Dataset/runs/classify/train2/val_batch1_pred.jpg (deflated 1%)
  adding: content/PlantVillage-Dataset/runs/classify/train2/val_batch0_labels.jpg (deflated 1%)
  adding: content/PlantVillage-Dataset/runs/classify/train2/val_batch1_labels.jpg (deflated 1%)
  adding: content/PlantVillage-Dataset/runs/classify/train2/val_batch2_labels.jpg (deflated 1%)
  adding: content/PlantVillage-

In [ ]:
from google.colab import files
files.download("/content/model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## F1 Score

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

conf_matrix = metrics.confusion_matrix.matrix

# Extract tp, fp, fn for each class
# conf_matrix[i] is the number of true values of each class predicted as class i
tp = np.diag(conf_matrix)
fp = np.sum(conf_matrix, axis=1) - TP
fn = np.sum(conf_matrix, axis=0) - TP

# Compute Precision, Recall, and F1-score per class
if (tp + fp) == 0:
  precision = 1
else:
  precision = tp / (tp + fp)

if (tp + fn):
  recall = tp / (tp + fn)

if (precision + recall) == 0:
  f1 = 1
else:
  f1 = 2 * (precision * recall) / (precision + recall)

# Compute macro & weighted F1-score
macro_f1 = np.mean(f1_per_class)
weights = np.sum(conf_matrix, axis=1)  # Number of samples per class
weighted_f1 = np.sum(f1_per_class * weights) / np.sum(weights)

# Print results
print(f"Macro F1-Score: {macro_f1:.4f}")
print(f"Weighted F1-Score: {weighted_f1:.4f}")


Macro F1-Score: 0.9491
Weighted F1-Score: 0.9500
